# 💰 Imports


In [1]:
import pandas as pd 
import numpy as np 
import dspy 
from dotenv import load_dotenv
import os 
import requests
from dspy.teleprompt import BayesianSignatureOptimizer
from dspy.evaluate.metrics import answer_exact_match
load_dotenv("../.env")



# CONFIG
random_state = 42
txt_to_idx = {
    'Clicnical Trials': '0',
    'pubmed': '1',
    'bioarxiv': '2',
    'chembl': '3'
}

NUM_THREADS = 5
DEV_NUM = 50

idx_to_txt = {v: k for k, v in txt_to_idx.items()}


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dspy setting up 

In [2]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key = os.environ["OPENAPI_KEY"])
# rm module is currently not available.
dspy.settings.configure(lm=turbo)

## 🎰 preprocessing

In [3]:
def get_router_source(x): 
    """
    Generates the routing label current source don't have proper labels we have appended disease_type + _ + pubmed for pubmed data
    fixing the above to return standardized source. 

    Args:
        x (str): standardized source string
    """
    if x.split("_")[-1] == "pubmed":
        return "pubmed"
    else:
        return x

df = pd.read_csv("../evaluation/question_answers_eval.csv")
df['route_option'] = df['Source'].apply(lambda x: get_router_source(x))

In [4]:
chembl_sample = df[df['route_option'] == 'chembl'].sample(n=49, random_state=random_state)
pubmed_sample = df[df['route_option'] == 'pubmed'].sample(n=49, random_state=random_state)
clinical_trials = df[df['route_option'] == 'Clicnical Trials'].sample(n=49, random_state=random_state)
bioarxiv_sample = df[df['route_option'] == 'bioarxiv'].sample(n=49, random_state=random_state)

In [21]:
all_samples = pd.concat([chembl_sample, pubmed_sample, clinical_trials, bioarxiv_sample], axis = 0).sample(frac = 1)
total_data = []
for idx, row in enumerate(all_samples.iterrows()): 
    dspy_example = dspy.Example({'question': row[1]["Question"], 'answer' : txt_to_idx[row[1]['route_option']]}).with_inputs("question")
    total_data.append(dspy_example)

# ⨍ Initialization

In [22]:
# setting up and testing the basic signature
class QA(dspy.Signature):
    question = dspy.InputField(desc = "Question to be routed to route")
    answer = dspy.OutputField(desc="route option should be integer only no additional text")

# testing out QA 
generate_answer = dspy.Predict(QA)
dev_example = total_data[0]
# Call the predictor on a particular input.
pred = generate_answer(question=dev_example.question )

# Print the input and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: How does the knockout of Prx4 from endothelial cells affect vulnerability to ischemia/reperfusion (I/R) injury in mice?
Predicted Answer: 1


In [23]:
dev_example.answer

'1'

In [24]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: Question to be routed to route
Answer: route option should be integer only no additional text

---

Question: How does the knockout of Prx4 from endothelial cells affect vulnerability to ischemia/reperfusion (I/R) injury in mice?
Answer: 1





In [25]:
class Router_module(dspy.Module):
    f"""
    Routes the specific question to relavant service we have following services as option {txt_to_idx}
    """
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(QA)
    
    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return dspy.Prediction(answer=prediction.answer)

### ⊹ Metrics definition 

In [26]:
def metric(gold, pred, trace = None ):
    actual_answer , pred_answer = gold.answer , pred.answer
    print('actual_answer', actual_answer) 
    print('predicted_answer', pred_answer)
    answer_EM = dspy.evaluate.answer_exact_match(gold,pred)
    return answer_EM


## 💉 BootstrapFewShot 

In [27]:
from dspy.teleprompt import BootstrapFewShot
teleprompter = BootstrapFewShot(metric=metric)
# Compile!
compiled_rag = teleprompter.compile(Router_module(), trainset=total_data[:DEV_NUM])

  2%|▏         | 1/50 [00:01<01:09,  1.42s/it]

actual_answer 1
predicted_answer 2


  4%|▍         | 2/50 [00:23<11:04, 13.84s/it]

actual_answer 1
predicted_answer 2


  6%|▌         | 3/50 [00:45<13:35, 17.35s/it]

...

actual_answer 1
predicted_answer 2


 20%|██        | 10/50 [05:06<25:23, 38.09s/it]

actual_answer 0
predicted_answer 0


 22%|██▏       | 11/50 [05:28<19:24, 29.85s/it]

actual_answer 0
predicted_answer 0
Bootstrapped 4 full traces after 12 examples in round 0.


In [28]:
compiled_rag

generate_answer = ChainOfThought(Signature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'Question to be routed to route', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'route option should be integer only no additional text', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [30]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: Question to be routed to route
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: route option should be integer only no additional text

---

Question: What is the purpose of the study Ilomedin Treatment for Patients Having Undergone Primary Percutaneous Coronary Intervention (PCI) ?
Answer: 0

---

Question: What is the need for constructing a novel TMSD based electrochemical detection strategy according to the research paper?
Answer: 1

---

Question: How do mineral composition, surface topography, and surface charge influence DNA adsorption behavior and preservation on mineral substrates?
Answer: 2

---

Question: Who is not recommending this work to be cited as a reference for projects?
Answer: 2

---

Question: Where did the study Safety and Tolerability of RNS60 Given by IV to Healthy Subjects take place
Answer: 0

---

Question: In what 

In [37]:
compiled_rag.save("router_bootstrapped_12_eg.json")

## 🪧 BayesianSignature

In [38]:
# BayesianSignature optimizer is not working 
# teleprompter_baysian = BayesianSignatureOptimizer(prompt_model="Route the respective question to the required service", task_model="route integer option", metric=metric, n=3)
# kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=0)
# compiled_router_prompt = teleprompter_baysian.compile(Router_module(),  devset=total_data[:DEV_NUM], optuna_trials_num=100, max_bootstrapped_demos=1, max_labeled_demos=2, eval_kwargs=kwargs)


  2%|▏         | 1/50 [00:01<01:11,  1.46s/it]

actual_answer 2
predicted_answer The specific individual or organization should be named.


  4%|▍         | 2/50 [00:02<01:07,  1.41s/it]

actual_answer 2
predicted_answer 3


  6%|▌         | 3/50 [00:24<08:15, 10.55s/it]

...

  6%|▌         | 3/50 [01:05<17:05, 21.82s/it]

actual_answer 0
predicted_answer 1


  8%|▊         | 4/50 [01:06<12:48, 16.71s/it]

actual_answer 1
predicted_answer 1
Bootstrapped 1 full traces after 5 examples in round 0.


TypeError: 'str' object is not callable

In [39]:
from dspy.teleprompt.signature_opt_typed import optimize_signature
from dspy.evaluate.metrics import answer_exact_match
from dspy.functional import TypedChainOfThought

compiled_program = optimize_signature(
    student=TypedChainOfThought("question -> answer"),
    evaluator=Evaluate(devset=total_data[:DEV_NUM], metric=answer_exact_match, num_threads=10, display_progress=True),
    n_iterations=50,
).program

ModuleNotFoundError: No module named 'dspy.teleprompt.signature_opt_typed'

## 🛥️ Bootstrap few shot with optuna

In [46]:
from dspy.teleprompt import BootstrapFewShotWithOptuna

fewshot_optuna_optimizer = BootstrapFewShotWithOptuna(metric=metric, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=NUM_THREADS)

your_dspy_program_compiled = fewshot_optuna_optimizer.compile(student=Router_module(),max_demos = 2,  trainset=total_data[:DEV_NUM], valset=total_data[DEV_NUM: DEV_NUM + 5])

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 8 candidate sets.


 14%|█▍        | 7/50 [00:00<00:00, 334.19it/s]
[I 2024-03-12 19:53:33,909] A new study created in memory with name: no-name-e0f37a3a-69ec-448e-bba4-73dbd777928f


actual_answer 1
predicted_answer 2
actual_answer 1
predicted_answer 2
actual_answer 2
predicted_answer 2
actual_answer 3
predicted_answer 2
actual_answer 1
predicted_answer 2
actual_answer 3
predicted_answer 2
actual_answer 2
predicted_answer 2
Bootstrapped 2 full traces after 8 examples in round 0.


Average Metric: 0 / 1  (0.0):  20%|██        | 1/5 [00:01<00:04,  1.14s/it]

actual_answer 3
predicted_answer 1


...

Error for example in dev set: 		 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-O0hnOySDX6ccyh9Epmb8cENl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Error for example in dev set: 		 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-O0hnOySDX6ccyh9Epmb8cENl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


Average Metric: 0.0 / 4  (0.0):  80%|████████  | 4/5 [17:43<03:01, 181.70s/it]   

Error for example in dev set: 		 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-O0hnOySDX6ccyh9Epmb8cENl on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Error for example in dev set: 		 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-O0hnOySDX6ccyh9Epmb8cENl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceed

Average Metric: 0.0 / 5  (0.0): 100%|██████████| 5/5 [19:29<00:00, 233.92s/it]
[I 2024-03-12 21:28:27,540] Trial 7 finished with value: 0.0 and parameters: {'demo_index_for_generate_answer': 1}. Best is trial 0 with value: 60.0.


actual_answer 3
predicted_answer 1
Average Metric: 0.0 / 5  (0.0%)
Best score: 60.0
Best program: generate_answer = ChainOfThought(Signature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'Question to be routed to route', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'route option should be integer only no additional text', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))


In [48]:
your_dspy_program_compiled.save("router_bootstrapped_few_shot_optuna.json")

In [49]:
turbo.inspect_history(n =1 )





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: Question to be routed to route
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: route option should be integer only no additional text

---

Question: What is the role of CRISPR-Cas systems in Enterococcus faecalis strains?
Reasoning: Let's think step by step in order to Answer: 2
Answer: 2

---

Question: How many aromatic rings are present in the structure of Lisinopril?
Reasoning: Let's think step by step in order to Answer: 1
Answer: 1



